# Example - viewing the results of a SAP calculation

This notebook views the results of a SAP10 calculation.

This uses the [sap10calcs Python package](https://github.com/stevenkfirth/sap10calcs).

## 1. Setup

In [51]:
import json
import matplotlib.pyplot as plt
import calendar
import pandas as pd
import sap10calcs

## 2. Run SAP calculation

In [52]:
result = sap10calcs.calculate(
    input_file = 'Example - creating a complete SAP XML file.xml',
    auth_token = None
)
result['sap_calculation_success']

True

## 3. Save the results

In [53]:
with open('Example - running a SAP calculation.json', 'w') as f:
    json.dump(result, f, indent = 4)

## 4. View results - initial look

In [54]:
# extract the SAP calculation results from the API information.
output_dict = result.pop('sap_calculation_output_dict')

In [55]:
# API information
result

{'api_call_datetime': '2024-11-08 14:40:16.686471',
 'api_call_url': 'https://netzeroapis.com/calc/sap10?calculation_method=Energy%20rating',
 'api_call_filename': 'Example - creating a complete SAP XML file.xml',
 'api_call_server_time_seconds': 0.023374629206955433,
 'licenses': ['https://netzeroapis.com/redoc#section/Terms-and-Conditions',
  'https://creativecommons.org/publicdomain/zero/1.0/'],
 'sap_calculation_success': True,
 'sap_calculation_error_type': None,
 'sap_calculation_error_message': None,
 'sap_calculation_error_traceback': None}

In [56]:
# SAP calculation results
print(type(output_dict))
print('length: ', len(output_dict))
print('first 20 items: ', {k:output_dict[k] for k in list(output_dict)[:20]})
print('last 20 items: ', {k:output_dict[k] for k in list(output_dict)[-20:]})

<class 'dict'>
length:  2345
first 20 items:  {'calculation_type': 'Energy rating', 'region_code': 0, 'postcode': None, 'postcode_area': None, 'postcode_district': None, 'latitude': 53.5, 'pcdb_fuel_price_table_name': None, 'main_heating_1_fghrs_index_number': None, 'value_1_building_part_1_level_0': 100.0, 'value_2_building_part_1_level_0': 3.0, 'value_3_level_0': 300.0, 'value_4': 100.0, 'value_5': 300.0, 'value_6a_total': 0, 'value_6a': 0.0, 'value_6b_total': 0, 'value_6b': 0.0, 'value_6c_total': 0, 'value_6c': 0.0, 'value_6d_total': 0}
last 20 items:  {'water_heating_other_fuel_price': 3.64, 'value_247': 112.22033408869557, 'electricity_fuel_code': 30, 'electricity_unit_price': 16.49, 'value_247a': 0.0, 'value_248': None, 'value_249': 0.0, 'value_250': 87.040629370826, 'value_251': 92, 'electricity_sold_to_grid_pv_unit_price': 5.59, 'electricity_sold_to_grid_other_unit_price': 5.59, 'value_252': 0.0, 'value_253': None, 'value_254': None, 'value_255': 648.001974813114, 'value_256': 

## 4. Results by SAP Worksheet section

This section shows the results as they are presented in the SAP Worksheet - see page 131 of the main SAP10.2 PDF document.

### SAP Worksheet - 1. Overall dwelling dimensions

In [6]:
pd.DataFrame(
    index = [x[8:] for x in output_dict if x.startswith('value_1_')],
    data = [{'Area (m2)': output_dict[x], 'Average storey height (m)': output_dict[x.replace('value_1', 'value_2')]} 
            for x in output_dict if x.startswith('value_1_')]
)

,Area (m2),Average storey height (m)
building_part_1_level_0,100.0,3.0


In [7]:
pd.DataFrame(
    index = [x[8:] for x in output_dict if x.startswith('value_3_')],
    data = [{'Volume (m3)': output_dict[x]} for x in output_dict if x.startswith('value_3_')]
)

,Volume (m3)
level_0,300.0


In [8]:
print('Total floor area: ', output_dict['value_4'], '(m2)')
print('Dwelling volume: ', output_dict['value_5'], '(m3)')

Total floor area:  100.0 (m2)
Dwelling volume:  300.0 (m3)


### SAP Worksheet - 2. Ventilation rate

In [9]:
pd.DataFrame(
    index = ['Number of chimneys / flues: - open chimneys', 'Number of chimneys / flues: - open flues', 
             'Number of chimneys / flues: - chimneys / flues attached to closed fire',
             'Number of chimneys / flues: - flues attached to solid fuel boiler', 
             'Number of chimneys / flues: - flues attached to other heater',
             'Number of blocked chimneys', 'Number of intermittent extract fans',
             'Number of passive vents', 'Number of flueless gas fires'
             ],
    data = {'Total': [output_dict.get(f'value_{x}_total') for x in ['6a', '6b', '6c', '6d', '6e', '6f', '7a', '7b', '7c']],
            'm3 per hour': [output_dict.get(f'value_{x}') for x in ['6a', '6b', '6c', '6d', '6e', '6f', '7a', '7b', '7c']]}
)

,Total,m3 per hour
Number of chimneys / flues: - open chimneys,0,0.0
Number of chimneys / flues: - open flues,0,0.0
Number of chimneys / flues: - chimneys / flues attached to closed fire,0,0.0
Number of chimneys / flues: - flues attached to solid fuel boiler,0,0.0
Number of chimneys / flues: - flues attached to other heater,0,0.0
Number of blocked chimneys,0,0.0
Number of intermittent extract fans,0,0.0
Number of passive vents,0,0.0
Number of flueless gas fires,0,0.0


In [10]:
print('Infiltration due to chimneys, flues, fans, PSVs, etc.: ', output_dict['value_8'], '(ach)')
print('Number of storeys in the dwelling: ', output_dict['value_9'])
print('Additional infiltration: ', output_dict['value_10'], '(ach)')
print('Structural infiltration: ', output_dict['value_11'], '(ach)')
print('If suspended wooden ground floor: ', output_dict['value_12'], '(ach)')
print('If no draught lobby: ', output_dict['value_13'], '(ach)')
print('Percentage of windows and doors draught proofed: ', output_dict['value_14'])
print('Window infiltration: ', output_dict['value_15'], '(ach)')
print('Infiltration rate (if no pressurisation test): ', output_dict['value_16'], '(ach)')
print('Air permeability value, AP50: ', output_dict['value_17'], '(m3/h/m2)')
print('Air permeability value, AP4: ', output_dict['value_17a'], '(m3/h/m2)')
print('Infiltration rate: ', output_dict['value_18'], '(ach)')
print('Number of sides on which dwelling is sheltered: ', output_dict['value_19'])
print('Shelter factor: ', output_dict['value_20'])
print('Infiltration rate incorporating shelter factor: ', output_dict['value_21'], '(ach)')

Infiltration due to chimneys, flues, fans, PSVs, etc.:  0.0 (ach)
Number of storeys in the dwelling:  None
Additional infiltration:  None (ach)
Structural infiltration:  None (ach)
If suspended wooden ground floor:  None (ach)
If no draught lobby:  None (ach)
Percentage of windows and doors draught proofed:  None
Window infiltration:  None (ach)
Infiltration rate (if no pressurisation test):  None (ach)
Air permeability value, AP50:  3.5 (m3/h/m2)
Air permeability value, AP4:  None (m3/h/m2)
Infiltration rate:  0.175 (ach)
Number of sides on which dwelling is sheltered:  2
Shelter factor:  0.85
Infiltration rate incorporating shelter factor:  0.14875 (ach)


In [11]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Monthly average wind speed (m/s)': [output_dict[f'value_22_{m}'] for m in range(1,13)],
        'Wind factor': [output_dict[f'value_22a_{m}'] for m in range(1,13)],
        'Adjusted infiltration rate (allowing for shelter and wind speed) (ach)': [output_dict[f'value_22b_{m}'] for m in range(1,13)],
    }
).round(2).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Monthly average wind speed (m/s),5.10,5.00,4.90,4.40,4.30,3.80,3.80,3.70,4.00,4.30,4.50,4.70
Wind factor,1.27,1.25,1.23,1.10,1.08,0.95,0.95,0.92,1.00,1.08,1.12,1.18
Adjusted infiltration rate (allowing for shelter and wind speed) (ach),0.19,0.19,0.18,0.16,0.16,0.14,0.14,0.14,0.15,0.16,0.17,0.17


In [12]:
print('If mechanical ventilation - air change rate through system: ', output_dict['value_23a'], '(ach)')
print('If exhaust air heat pump using Appendix N - air change rate through system: ', output_dict['value_23b'], '(ach)')
print('If balanced with heat recovery - air change rate through system: ', output_dict['value_23c'], '(ach)')

If mechanical ventilation - air change rate through system:  None (ach)
If exhaust air heat pump using Appendix N - air change rate through system:  None (ach)
If balanced with heat recovery - air change rate through system:  None (ach)


In [13]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'If balanced mechanical ventilation with heat recovery (MVHR) (ach)': [output_dict[f'value_24a_{m}'] for m in range(1,13)],
        'If balanced mechanical ventilation without heat recovery (MV) (ach)': [output_dict[f'value_24b_{m}'] for m in range(1,13)],
        'If whole house extract ventilation or positive input ventilation from outside (ach)': [output_dict[f'value_24c_{m}'] for m in range(1,13)],
        'If natural ventilation or whole house positive input ventilation from loft (ach)': [output_dict[f'value_24d_{m}'] for m in range(1,13)],
        'Effective air change rate (ach)': [output_dict[f'value_25_{m}'] for m in range(1,13)]
    }
).round(2).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
If balanced mechanical ventilation with heat recovery (MVHR) (ach),None,None,None,None,None,None,None,None,None,None,None,None
If balanced mechanical ventilation without heat recovery (MV) (ach),None,None,None,None,None,None,None,None,None,None,None,None
If whole house extract ventilation or positive input ventilation from outside (ach),None,None,None,None,None,None,None,None,None,None,None,None
If natural ventilation or whole house positive input ventilation from loft (ach),0.52,0.52,0.52,0.51,0.51,0.51,0.51,0.51,0.51,0.51,0.51,0.52
Effective air change rate (ach),0.52,0.52,0.52,0.51,0.51,0.51,0.51,0.51,0.51,0.51,0.51,0.52


### SAP Worksheet - 3. Heat losses and heat loss parameter

In [14]:
pd.DataFrame(
    index = [x[:-9] for x in output_dict if x.endswith('net_area')],
    data = [{'Gross area (m2)': output_dict.get(x.replace('net_area', 'gross_area')),
             'Openings (m2)': output_dict.get(x.replace('net_area', 'openings_area')),
             'Net area (m2)': output_dict[x], 
             'U-value (W/m2K)': output_dict[x.replace('net_area', 'u_value')],
             'k-value': output_dict.get(x.replace('net_area', 'kappa_value'))} 
            for x in output_dict if x.endswith('net_area')]
)

,Gross area (m2),Openings (m2),Net area (m2),U-value (W/m2K),k-value
window_1,NaN,NaN,7.5,1.415094,NaN
ground_floor_1,NaN,NaN,100.0,0.500000,0.0
external_wall_1,120.0,7.5,112.5,0.500000,0.0
roof_1,100.0,0.0,100.0,0.500000,0.0


In [15]:
l = ['26', '26a', '27', '27a', '28', '28a', '28b', '29', '29a', '30', '32', '32a', '32b', '32c', '32d', '32e']
pd.DataFrame(
    index = ['Solid door', 'Semi-glazed door', 'Window', 'Roof window', 'Basement floor', 'Ground floor', 'Exposed floor',
             'Basement wall', 'External wall', 'Roof', 'Party wall', 'Party floor', 'Party ceiling', 'Internal wall', 
             'Internal floor', 'Internal ceiling'],
    data = {'A * U (W/K)': [output_dict.get(f'value_{x}_UA') for x in l],
            'A * k (W/K)': [output_dict.get(f'value_{x}_Uk') for x in l]}  
)

,A * U (W/K),A * k (W/K)
Solid door,0.000000,None
Semi-glazed door,0.000000,None
Window,10.613208,None
Roof window,0.000000,None
Basement floor,0.000000,None
Ground floor,50.000000,None
Exposed floor,0.000000,None
Basement wall,0.000000,None
External wall,56.250000,None
Roof,50.000000,None


In [16]:
print('Total area of external elements: ', output_dict['value_31'], '(m2)')
print('Fabric heat loss: ', output_dict['value_33'], '(W/K)')
print('Heat capcity: ', output_dict['value_34'], '(kJ/K)')
print('Thermal mass parameter', output_dict['value_35'], '(kJ/m2K)')
print('Linear Thermal bridges: ', output_dict['value_36'], '(W/K)')
print('Point Thermal bridges: ', output_dict['value_36a'], '(W/K)')
print('Total fabric heat loss: ', output_dict['value_37'], '(W/K)')

Total area of external elements:  320.0 (m2)
Fabric heat loss:  166.8632075471698 (W/K)
Heat capcity:  0.0 (kJ/K)
Thermal mass parameter 0.0 (kJ/m2K)
Linear Thermal bridges:  21.5 (W/K)
Point Thermal bridges:  0 (W/K)
Total fabric heat loss:  188.3632075471698 (W/K)


In [17]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Ventilation heat loss calculated monthly (W/K)': [output_dict[f'value_38_{m}'] for m in range(1,13)],
        'Heat transfer coefficient (W/K)': [output_dict[f'value_39_{m}'] for m in range(1,13)],
        'Heat loss parameter (W/m2K)': [output_dict[f'value_40_{m}'] for m in range(1,13)],
        'Number of days in month': [output_dict[f'value_41_{m}'] for m in range(1,13)]
    }
).round(2).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Ventilation heat loss calculated monthly (W/K),51.28,51.21,51.14,50.83,50.77,50.49,50.49,50.44,50.60,50.77,50.89,51.01
Heat transfer coefficient (W/K),239.64,239.57,239.51,239.19,239.13,238.85,238.85,238.80,238.96,239.13,239.25,239.38
Heat loss parameter (W/m2K),2.40,2.40,2.40,2.39,2.39,2.39,2.39,2.39,2.39,2.39,2.39,2.39
Number of days in month,31.00,28.00,31.00,30.00,31.00,30.00,31.00,31.00,30.00,31.00,30.00,31.00


In [18]:
print('Heat transfer coefficient: ', output_dict['value_39'], '(W/K)')
print('Heat loss parameter: ', output_dict['value_40'], '(W/m2K)')

Heat transfer coefficient:  239.18819278288757 (W/K)
Heat loss parameter:  2.3918819278288757 (W/m2K)


### SAP Worksheet - 4. Water heating energy requirement

In [19]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Hot water usage in litres per day for mixer showers': [output_dict[f'value_42a_{m}'] for m in range(1,13)],
        'Hot water usage in litres per day for baths': [output_dict[f'value_42b_{m}'] for m in range(1,13)],
        'Hot water usage in litres per day for other uses': [output_dict[f'value_42c_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Hot water usage in litres per day for mixer showers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hot water usage in litres per day for baths,85.2,83.9,82.1,78.8,76.4,73.7,72.2,74.0,75.9,78.8,82.1,84.9
Hot water usage in litres per day for other uses,44.9,43.3,41.7,40.0,38.4,36.8,36.8,38.4,40.0,41.7,43.3,44.9


In [20]:
print('Annual average hot water usage in litres per day: ', output_dict['value_43'])

Annual average hot water usage in litres per day:  119.81494745334464


In [21]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Hot water usage in litres per day for each month': [output_dict[f'value_44_{m}'] for m in range(1,13)],
        'Energy content of hot water used (kWh)': [output_dict[f'value_45_{m}'] for m in range(1,13)],
        'Distribution loss (kWh)': [output_dict[f'value_46_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Hot water usage in litres per day for each month,130.1,127.2,123.8,118.9,114.8,110.4,108.9,112.3,115.9,120.5,125.4,129.8
Energy content of hot water used (kWh),206.1,181.1,190.3,162.7,154.5,135.8,131.8,139.1,142.9,163.5,178.7,203.3
Distribution loss (kWh),30.9,27.2,28.5,24.4,23.2,20.4,19.8,20.9,21.4,24.5,26.8,30.5


In [22]:
print('Energy content of hot water used', output_dict['value_45'], '(kWh)')
print('Storage volume (litres) including any solar or WWHRS storage within same vessel: ', output_dict['value_47'])
print('Water storage loss (or HIU loss) - If manufacturer’s declared loss factor is known: ', output_dict['value_48'], '(kWh/day)')
print('Temperature factor - If manufacturer’s declared loss factor is known: ', output_dict['value_49'])
print('Energy lost from water storage - If manufacturer’s declared loss factor is known: ', output_dict['value_50'], '(kWh/day)')
print('Hot water storage loss factor - If manufacturer’s declared loss factor is not known: ', output_dict['value_51'], '(kWh/day)')
print('Volume factor - If manufacturer’s declared loss factor is not known: ', output_dict['value_52'])
print('Temperature factor - If manufacturer’s declared loss factor is not known: ', output_dict['value_53'])
print('Energy lost from water storage - If manufacturer’s declared loss factor is not known: ', output_dict['value_54'], '(kWh/day)')
print('Energy lost from water storage: ', output_dict['value_55'], '(kWh/day)')

Energy content of hot water used 1989.7000174314362 (kWh)
Storage volume (litres) including any solar or WWHRS storage within same vessel:  0
Water storage loss (or HIU loss) - If manufacturer’s declared loss factor is known:  None (kWh/day)
Temperature factor - If manufacturer’s declared loss factor is known:  None
Energy lost from water storage - If manufacturer’s declared loss factor is known:  None (kWh/day)
Hot water storage loss factor - If manufacturer’s declared loss factor is not known:  None (kWh/day)
Volume factor - If manufacturer’s declared loss factor is not known:  None
Temperature factor - If manufacturer’s declared loss factor is not known:  None
Energy lost from water storage - If manufacturer’s declared loss factor is not known:  None (kWh/day)
Energy lost from water storage:  0 (kWh/day)


In [23]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Water storage (or HIU) loss calculated for each month (kWh)': [output_dict[f'value_56_{m}'] for m in range(1,13)],
        'If the vessel contains dedicated solar storage or dedicated WWHRS storage': [output_dict[f'value_57_{m}'] for m in range(1,13)],
        'Primary circuit loss for each month (kWh)': [output_dict[f'value_59_{m}'] for m in range(1,13)],
        'Combi loss for each month (kWh)': [output_dict[f'value_61_{m}'] for m in range(1,13)],
        'Total heat required for water heating calculated for each month (kWh)': [output_dict[f'value_62_{m}'] for m in range(1,13)],
        'WWHRS DHW input (kWh)': [output_dict[f'value_63a_{m}'] for m in range(1,13)],
        'PV diverter DHW input (kWh)': [output_dict[f'value_63b_{m}'] for m in range(1,13)],
        'Solar DHW input (kWh)': [output_dict[f'value_63c_{m}'] for m in range(1,13)],
        'FGHRS DHW input (kWh)': [output_dict[f'value_63d_{m}'] for m in range(1,13)],
        'Output from water heater for each month (kWh)': [output_dict[f'value_64_{m}'] for m in range(1,13)],
        'Energy used by instantaneous electric shower(s) (kWh)': [output_dict[f'value_64a_{m}'] for m in range(1,13)],
        'Heat gains from water heating (kWh)': [output_dict[f'value_65_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Water storage (or HIU) loss calculated for each month (kWh),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
If the vessel contains dedicated solar storage or dedicated WWHRS storage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Primary circuit loss for each month (kWh),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Combi loss for each month (kWh),51.0,46.0,51.0,49.3,51.0,49.3,51.0,51.0,49.3,51.0,49.3,51.0
Total heat required for water heating calculated for each month (kWh),257.0,227.2,241.2,212.0,205.5,185.1,182.7,190.1,192.2,214.4,228.0,254.2
WWHRS DHW input (kWh),-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
PV diverter DHW input (kWh),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Solar DHW input (kWh),-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
FGHRS DHW input (kWh),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Output from water heater for each month (kWh),257.0,227.2,241.2,212.0,205.5,185.1,182.7,190.1,192.2,214.4,228.0,254.2


In [24]:
print('Energy used by instantaneous electric shower(s): ', output_dict['value_64a'], '(kWh)')

Energy used by instantaneous electric shower(s):  0 (kWh)


### SAP Worksheet - 5. Internal gains

In [25]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Metabolic gains (W)': [output_dict[f'value_66_{m}'] for m in range(1,13)],
        'Lighting gains (W)': [output_dict[f'value_67_{m}'] for m in range(1,13)],
        'Appliances gains (W)': [output_dict[f'value_68_{m}'] for m in range(1,13)],
        'Cooking gains (W)': [output_dict[f'value_69_{m}'] for m in range(1,13)],
        'Pumps and fans gains (W)': [output_dict[f'value_70_{m}'] for m in range(1,13)],
        'Losses e.g. evaporation (W)': [output_dict[f'value_71_{m}'] for m in range(1,13)],
        'Water heating gains (W)': [output_dict[f'value_72_{m}'] for m in range(1,13)],
        'Total internal gains (W)': [output_dict[f'value_73_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Metabolic gains (W),164.4,164.4,164.4,164.4,164.4,164.4,164.4,164.4,164.4,164.4,164.4,164.4
Lighting gains (W),74.7,66.4,54.0,40.9,30.5,25.8,27.9,36.2,48.6,61.7,72.0,76.8
Appliances gains (W),382.6,386.5,376.5,355.2,328.4,303.1,286.2,282.2,292.2,313.5,340.4,365.7
Cooking gains (W),54.2,54.2,54.2,54.2,54.2,54.2,54.2,54.2,54.2,54.2,54.2,54.2
Pumps and fans gains (W),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Losses e.g. evaporation (W),-109.6,-109.6,-109.6,-109.6,-109.6,-109.6,-109.6,-109.6,-109.6,-109.6,-109.6,-109.6
Water heating gains (W),109.2,106.7,102.2,92.3,86.2,79.8,76.0,79.3,83.1,90.2,99.7,108.0
Total internal gains (W),675.5,668.6,641.6,597.3,554.1,517.7,499.1,506.7,533.0,574.4,621.1,659.4


### SAP Worksheet - 6. Solar gains

In [26]:
# solar radiation on the horizontal plane
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'solar_radiation_on_horizontal_plane': [output_dict[f'solar_radiation_on_horizontal_plane_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
solar_radiation_on_horizontal_plane,26,54,96,150,192,200,189,157,115,66,33,21


In [27]:
# access factors, area, transmittance and frame factor
pd.DataFrame(
    index = [x[:-20] for x in output_dict if x.endswith('solar_transmittance')],
    data = [{'Winter solar access factor': output_dict.get(x.replace('solar_transmittance', 'winter_solar_access_factor')),
             'Summer solar access factor': output_dict.get(x.replace('solar_transmittance', 'summer_solar_access_factor')),
             'Area (m2)': output_dict.get(x.replace('solar_transmittance', 'window_area')),
             'Solar transmittance': output_dict.get(x),
             'Frame factor': output_dict.get(x.replace('solar_transmittance', 'frame_factor')),} 
            for x in output_dict if x.endswith('solar_transmittance')]
)

,Winter solar access factor,Summer solar access factor,Area (m2),Solar transmittance,Frame factor
window_0,0.77,0.9,7.5,0.684,0.7


In [28]:
# factor for converting horizontal to vertical (R_h_inc), solar flux (W/m2) and solar gains (W)
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        f'{x[:-20]}_{y[0]} {y[1]}': [output_dict.get(f"{x.replace('solar_transmittance', y[0])}_{m}") for m in range(1,13)] 
        for x in output_dict if x.endswith('solar_transmittance') 
        for y in [('R_h_inc',''), ('solar_flux','(W/m2)'), ('solar_gains', '(W)')]
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
window_0_R_h_inc,NaN,1.4,1.0,0.7,0.6,0.6,0.6,0.7,0.9,1.3,1.7,1.9
window_0_solar_flux (W/m2),46.8,76.6,97.5,110.2,114.9,110.5,108.0,104.9,101.9,82.6,55.4,40.4
window_0_solar_gains (W),116.3,190.5,242.7,274.3,285.9,275.1,268.8,261.0,253.5,205.5,137.9,100.5


In [29]:
# Solar gains by orientation (W)
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        #'solar_radiation_on_horizontal_plane': [output_dict[f'solar_radiation_on_horizontal_plane_{m}'] for m in range(1,13)],
        'North': [output_dict[f'value_74_{m}'] for m in range(1,13)], 
        'Northeast': [output_dict[f'value_75_{m}'] for m in range(1,13)], 
        'East': [output_dict[f'value_76_{m}'] for m in range(1,13)], 
        'Southeast': [output_dict[f'value_77_{m}'] for m in range(1,13)], 
        'South': [output_dict[f'value_78_{m}'] for m in range(1,13)], 
        'Southwest': [output_dict[f'value_79_{m}'] for m in range(1,13)], 
        'West': [output_dict[f'value_80_{m}'] for m in range(1,13)],
        'Northwest': [output_dict[f'value_81_{m}'] for m in range(1,13)], 
        'Roof windows': [output_dict[f'value_82_{m}'] for m in range(1,13)], 
        'Rooflight': [output_dict[f'value_82a_{m}'] for m in range(1,13)]
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Northeast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Southeast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
South,116.3,190.5,242.7,274.3,285.9,275.1,268.8,261.0,253.5,205.5,137.9,100.5
Southwest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Northwest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Roof windows,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rooflight,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# solar gains and total gains
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Solar gains (W)': [output_dict[f'value_83_{m}'] for m in range(1,13)],
        'Total gains – internal and solar (W)': [output_dict[f'value_84_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Solar gains (W),116.3,190.5,242.7,274.3,285.9,275.1,268.8,261.0,253.5,205.5,137.9,100.5
Total gains – internal and solar (W),791.8,859.2,884.4,871.7,839.9,792.8,767.8,767.8,786.5,779.9,759.0,760.0


### SAP Worksheet - 7. Mean internal temperature (heating season)

In [31]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Utilisation factor for gains for living area': [output_dict[f'value_86_{m}'] for m in range(1,13)],
        'Mean internal temperature in living area (degC)': [output_dict[f'value_87_{m}'] for m in range(1,13)],
        'Temperature during heating periods in rest of dwelling (degC)': [output_dict[f'value_88_{m}'] for m in range(1,13)],
        'Utilisation factor for gains for rest of dwelling': [output_dict[f'value_89_{m}'] for m in range(1,13)],
        'Mean internal temperature in the rest of dwelling (degC)': [output_dict[f'value_90_{m}'] for m in range(1,13)],
        'Mean internal temperature (for the whole dwelling) (degC)': [output_dict[f'value_92_{m}'] for m in range(1,13)],
        'Adjusted mean internal temperature (degC)': [output_dict[f'value_93_{m}'] for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Utilisation factor for gains for living area,0.8,0.8,0.8,0.8,0.7,0.7,0.6,0.6,0.7,0.8,0.8,0.8
Mean internal temperature in living area (degC),14.6,15.0,15.7,16.7,17.9,19.1,19.8,19.8,18.9,17.4,15.8,14.5
Temperature during heating periods in rest of dwelling (degC),19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1,19.1
Utilisation factor for gains for rest of dwelling,0.8,0.8,0.8,0.7,0.7,0.6,0.4,0.5,0.6,0.7,0.8,0.8
Mean internal temperature in the rest of dwelling (degC),13.5,13.9,14.6,15.6,16.8,17.9,18.6,18.5,17.7,16.3,14.7,13.5
Mean internal temperature (for the whole dwelling) (degC),13.9,14.2,14.9,16.0,17.1,18.3,19.0,18.9,18.1,16.6,15.1,13.8
Adjusted mean internal temperature (degC),13.9,14.2,14.9,16.0,17.1,18.3,19.0,18.9,18.1,16.6,15.1,13.8


### SAP Worksheet - 8. Space heating requirement

In [32]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Utilisation factor for gains': [output_dict[f'value_94_{m}'] for m in range(1,13)],
        'Useful gains (W)': [output_dict[f'value_95_{m}'] for m in range(1,13)],
        'Monthly average external temperature (degC)': [output_dict[f'value_96_{m}'] for m in range(1,13)],
        'Heat loss rate for mean internal temperature (W)': [output_dict[f'value_97_{m}'] for m in range(1,13)],
        'Space heating requirement for each month (kWh)': [output_dict.get(f'value_98a_{m}') for m in range(1,13)],
        'Solar space heating (kWh)': [output_dict.get(f'value_98b_{m}') for m in range(1,13)],
        'Space heating requirement for each month after solar contribution (kWh)': [output_dict.get(f'value_98c_{m}') for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Utilisation factor for gains,0.7,0.7,0.7,0.7,0.6,0.5,0.4,0.4,0.5,0.6,0.7,0.8
Useful gains (W),588.5,620.4,615.4,575.3,509.9,415.4,325.3,335.6,429.2,505.4,542.8,570.8
Monthly average external temperature (degC),4.3,4.9,6.5,8.9,11.7,14.6,16.6,16.4,14.1,10.6,7.1,4.2
Heat loss rate for mean internal temperature (W),2292.0,2231.7,2023.3,1692.1,1297.4,872.7,564.5,596.1,944.9,1435.9,1905.1,2293.2
Space heating requirement for each month (kWh),1267.4,1082.9,1047.5,804.1,586.0,NaN,NaN,NaN,NaN,692.3,980.9,1281.5
Solar space heating (kWh),0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
Space heating requirement for each month after solar contribution (kWh),1267.4,1082.9,1047.5,804.1,586.0,NaN,NaN,NaN,NaN,692.3,980.9,1281.5


In [33]:
print('Space heating requirement: ', output_dict['value_98a'], '(kWh/year)')
print('Solar space heating: ', output_dict['value_98b'], '(kWh/year)')
print('Space heating requirement after solar contribution: ', output_dict['value_98c'], '(kWh/year)')
print('Space heating requirement per unit floor area: ',output_dict['value_99'], '(kWh/m2/year)')

Space heating requirement:  7742.4560156411535 (kWh/year)
Solar space heating:  0 (kWh/year)
Space heating requirement after solar contribution:  7742.4560156411535 (kWh/year)
Space heating requirement per unit floor area:  77.42456015641153 (kWh/m2/year)


### SAP Worksheet - 8c. Space cooling requirement

In [34]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Heat loss rate': [output_dict.get(f'value_100_{m}') for m in range(1,13)],
        'Utilisation factor for loss': [output_dict.get(f'value_101_{m}') for m in range(1,13)],
        'Useful loss (W)': [output_dict.get(f'value_102_{m}') for m in range(1,13)],
        'Gains (W)': [output_dict.get(f'value_103_{m}') for m in range(1,13)],
        'Space cooling requirement for month, whole dwelling, continuous (kWh)': [output_dict.get(f'value_104_{m}') for m in range(1,13)],
        'Intermittency factor': [output_dict.get(f'value_106_{m}') for m in range(1,13)],
        'Space cooling requirement for month (kWh)': [output_dict.get(f'value_107_{m}') for m in range(1,13)],
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Heat loss rate,None,None,None,None,None,None,None,None,None,None,None,None
Utilisation factor for loss,None,None,None,None,None,None,None,None,None,None,None,None
Useful loss (W),None,None,None,None,None,None,None,None,None,None,None,None
Gains (W),None,None,None,None,None,None,None,None,None,None,None,None
"Space cooling requirement for month, whole dwelling, continuous (kWh)",None,None,None,None,None,None,None,None,None,None,None,None
Intermittency factor,None,None,None,None,None,None,None,None,None,None,None,None
Space cooling requirement for month (kWh),None,None,None,None,None,None,None,None,None,None,None,None


In [35]:
print('Space cooling requirement, whole dwelling, continuous: ', output_dict.get('value_104'), '(kWh)')
print('Cooled fraction: ', output_dict.get('value_105'))

Space cooling requirement, whole dwelling, continuous:  None (kWh)
Cooled fraction:  None


### SAP Worksheet - 8f. Fabric Energy Efficiency

In [36]:
print('Fabric Energy Efficiency: ', output_dict.get('Value_109'))

Fabric Energy Efficiency:  None


### SAP Worksheet - 9a. Energy requirements – Individual heating systems including micro-CHP

In [37]:
print('Fraction of space heat from secondary/supplementary system: ', output_dict.get('value_201'))
print('Fraction of space heat from main system(s): ', output_dict.get('value_202'))
print('Fraction of main heating from main system 2: ', output_dict.get('value_203'))
print('Fraction of total space heat from main system 1: ', output_dict.get('value_204'))
print('Fraction of total space heat from main system 2: ', output_dict.get('value_205'))
print('Efficiency of main space heating system 1 (in %): ', output_dict.get('value_206'))
print('Efficiency of main space heating system 2 (in %): ', output_dict.get('value_207'))
print('Efficiency of secondary/supplementary heating system, %: ', output_dict.get('value_208'))
print('Cooling System Seasonal Energy Efficiency Ratio: ' , output_dict.get('value_209'))

Fraction of space heat from secondary/supplementary system:  0
Fraction of space heat from main system(s):  1.0
Fraction of main heating from main system 2:  0
Fraction of total space heat from main system 1:  1.0
Fraction of total space heat from main system 2:  0.0
Efficiency of main space heating system 1 (in %):  79
Efficiency of main space heating system 2 (in %):  None
Efficiency of secondary/supplementary heating system, %:  None
Cooling System Seasonal Energy Efficiency Ratio:  None


In [38]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Space heating requirement (kWh)': [output_dict.get(f'value_98c_{m}') for m in range(1,13)],
        'Space heating fuel (main heating system 1) (kWh)': [output_dict.get(f'value_211_{m}') for m in range(1,13)],
        'Space heating fuel (main heating system 2) (kWh)': [output_dict.get(f'value_213_{m}') for m in range(1,13)],
        'Space heating fuel (secondary) (kWh)': [output_dict.get(f'value_215_{m}') for m in range(1,13)],
        'Output from water heater (kWh)': [output_dict.get(f'value_64_{m}') for m in range(1,13)],
        'Efficiency of water heater': [output_dict.get(f'value_217_{m}') for m in range(1,13)],
        'Fuel for water heating (kWh)': [output_dict.get(f'value_219_{m}') for m in range(1,13)],
        'Space cooling fuel (kWh)': [output_dict.get(f'value_221_{m}') for m in range(1,13)]
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Space heating requirement (kWh),1267.4,1082.9,1047.5,804.1,586.0,NaN,NaN,NaN,NaN,692.3,980.9,1281.5
Space heating fuel (main heating system 1) (kWh),1604.3,1370.7,1326.0,1017.8,741.7,NaN,NaN,NaN,NaN,876.3,1241.6,1622.1
Space heating fuel (main heating system 2) (kWh),None,None,None,None,None,None,None,None,None,None,None,None
Space heating fuel (secondary) (kWh),None,None,None,None,None,None,None,None,None,None,None,None
Output from water heater (kWh),257.0,227.2,241.2,212.0,205.5,185.1,182.7,190.1,192.2,214.4,228.0,254.2
Efficiency of water heater,84.0,84.0,84.0,84.0,84.0,84.0,84.0,84.0,84.0,84.0,84.0,84.0
Fuel for water heating (kWh),306.0,270.4,287.2,252.4,244.6,220.3,217.5,226.3,228.9,255.3,271.5,302.6
Space cooling fuel (kWh),None,None,None,None,None,None,None,None,None,None,None,None


In [39]:
print('Fuel for water heating: ', output_dict.get('value_219'), '(kWh)')
print('Space cooling fuel: ', output_dict.get('value_221'), '(kWh)')
print('Space heating fuel used, main system 1: ', output_dict.get('value_211'), '(kWh)')
print('Space heating fuel used, main system 2: ', output_dict.get('value_213'), '(kWh)')
print('Space heating fuel used, secondary: ', output_dict.get('value_215'), '(kWh)')
print('Water heating fuel used: ', output_dict.get('value_219'), '(kWh)')
print('Electricity for instantaneous electric shower(s): ', output_dict.get('value_64a'), '(kWh)')
print('Space cooling fuel used: ', output_dict.get('value_221'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: mechanical ventilation fans - balanced, extract or positive input from outside: ', output_dict.get('value_230a'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: warm air heating system fans: ', output_dict.get('value_230b'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: heating circulation pump or water pump within warm air heating unit: ', output_dict.get('value_230c'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: oil boiler auxiliary (oil pump, flue fan, etc; excludes circulation pump): ', output_dict.get('value_230d'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: gas boiler auxiliary (flue fan, etc; excludes circulation pump): ', output_dict.get('value_230e'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: maintaining electric keep-hot facility for gas combi boiler: ', output_dict.get('value_230f'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: pump for solar water heating: ', output_dict.get('value_230g'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: pump for storage WWHRS: ', output_dict.get('value_230h'), '(kWh)')
print('Electricity for pumps, fans and electric keep-hot: Total electricity for the above: ', output_dict.get('value_231'), '(kWh)')
print('Electricity for lighting: ', output_dict.get('value_232'), '(kWh)')

Fuel for water heating:  3082.9762112279 (kWh)
Space cooling fuel:  None (kWh)
Space heating fuel used, main system 1:  9800.577234988803 (kWh)
Space heating fuel used, main system 2:  None (kWh)
Space heating fuel used, secondary:  None (kWh)
Water heating fuel used:  3082.9762112279 (kWh)
Electricity for instantaneous electric shower(s):  0 (kWh)
Space cooling fuel used:  None (kWh)
Electricity for pumps, fans and electric keep-hot: mechanical ventilation fans - balanced, extract or positive input from outside:  0 (kWh)
Electricity for pumps, fans and electric keep-hot: warm air heating system fans:  0 (kWh)
Electricity for pumps, fans and electric keep-hot: heating circulation pump or water pump within warm air heating unit:  0 (kWh)
Electricity for pumps, fans and electric keep-hot: oil boiler auxiliary (oil pump, flue fan, etc; excludes circulation pump):  0 (kWh)
Electricity for pumps, fans and electric keep-hot: gas boiler auxiliary (flue fan, etc; excludes circulation pump):  0

In [49]:
pd.DataFrame(
    index = [calendar.month_name[m][:3] for m in range(1,13)],
    data = {
        'Energy used in dwelling: Electricity generated by PVs (kWh)': [output_dict.get(f'value_233a_{m}') for m in range(1,13)],
        'Energy used in dwelling: Electricity generated by wind turbines (kWh)': [output_dict.get(f'value_234a_{m}') for m in range(1,13)],
        'Energy used in dwelling: Electricity generated by hydro-electric generators (kWh)': [output_dict.get(f'value_235a_{m}') for m in range(1,13)],
        'Energy used in dwelling: Electricity used or net electricity generated by micro-CHP (kWh)': [output_dict.get(f'value_235c_{m}') for m in range(1,13)],
        'Energy exported: Electricity generated by PVs (kWh)': [output_dict.get(f'value_233b_{m}') for m in range(1,13)],
        'Energy exported: Electricity generated by wind turbines (kWh)': [output_dict.get(f'value_234b_{m}') for m in range(1,13)],
        'Energy exported: Electricity generated by hydro-electric generators (kWh)': [output_dict.get(f'value_235b_{m}') for m in range(1,13)],
        'Energy exported: Electricity used or net electricity generated by micro-CHP (kWh)': [output_dict.get(f'value_235d_{m}') for m in range(1,13)]
    }
).round(1).T

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
Energy used in dwelling: Electricity generated by PVs (kWh),None,None,None,None,None,None,None,None,None,None,None,None
Energy used in dwelling: Electricity generated by wind turbines (kWh),0,0,0,0,0,0,0,0,0,0,0,0
Energy used in dwelling: Electricity generated by hydro-electric generators (kWh),0,0,0,0,0,0,0,0,0,0,0,0
Energy used in dwelling: Electricity used or net electricity generated by micro-CHP (kWh),0,0,0,0,0,0,0,0,0,0,0,0
Energy exported: Electricity generated by PVs (kWh),None,None,None,None,None,None,None,None,None,None,None,None
Energy exported: Electricity generated by wind turbines (kWh),0,0,0,0,0,0,0,0,0,0,0,0
Energy exported: Electricity generated by hydro-electric generators (kWh),0,0,0,0,0,0,0,0,0,0,0,0
Energy exported: Electricity used or net electricity generated by micro-CHP (kWh),0,0,0,0,0,0,0,0,0,0,0,0


!!! Should read None rather than 0 for wind, hydro and m-CHP

In [41]:
# Appendix Q
pd.DataFrame(
    index = [x for x in list(map(chr, range(97, 123))) if f'value_236{x}' in output_dict],
    data = [
        {
            'Appendix Q: energy saved or generated': output_dict.get(f'value_236{x}'),
            'Appendix Q: energy used': output_dict.get(f'value_237{x}')
        }
        for x in list(map(chr, range(97, 123))) if f'value_236{x}' in output_dict
    ]
    )


""


In [42]:
print('Total delivered energy for all uses: ', output_dict.get('value_238'), '(kWh)')

Total delivered energy for all uses:  13411.392314444878 (kWh)


### SAP Worksheet - 10a. Fuel costs – Individual heating systems including micro-CHP

In [62]:
print('Space heating - main system 1 (electric off-peak tariff): High-rate fraction: ', output_dict.get('value_240a'))
print('Space heating - main system 1 (electric off-peak tariff): Low-rate fraction: ', output_dict.get('value_240b'))
print('Space heating - main system 1 (electric off-peak tariff): High-rate cost: Fuel Price: ', output_dict.get('space_heating_main_system_1_high_rate_fuel_price'), '(£/kWh)')
print('Space heating - main system 1 (electric off-peak tariff): High-rate cost: Fuel Cost: ', output_dict.get('value_240c'), '(£/year)'),
print('Space heating - main system 1 (electric off-peak tariff): Low-rate cost: Fuel Price: ', output_dict.get('space_heating_main_system_1_low_rate_fuel_price'), '(£/kWh)')
print('Space heating - main system 1 (electric off-peak tariff): Low-rate cost: Fuel Cost: ', output_dict.get('value_240d'), '(£/year)')
print('Space heating - main system 1 (other fuel): Fuel Price: ', output_dict.get('space_heating_main_system_1_other_fuel_price'), '(£/kWh)')
print('Space heating - main system 1 (other fuel): Fuel Cost: ', '{:.2f}'.format(output_dict.get('value_240e')), '(£/year)')

Space heating - main system 1 (electric off-peak tariff): High-rate fraction:  None
Space heating - main system 1 (electric off-peak tariff): Low-rate fraction:  None
Space heating - main system 1 (electric off-peak tariff): High-rate cost: Fuel Price:  None (£/kWh)
Space heating - main system 1 (electric off-peak tariff): High-rate cost: Fuel Cost:  None (£/year)
Space heating - main system 1 (electric off-peak tariff): Low-rate cost: Fuel Price:  None (£/kWh)
Space heating - main system 1 (electric off-peak tariff): Low-rate cost: Fuel Cost:  None (£/year)
Space heating - main system 1 (other fuel): Fuel Price:  3.64 (£/kWh)
Space heating - main system 1 (other fuel): Fuel Cost:  356.74 (£/year)


In [44]:
print('Space heating - main system 2 (electric off-peak tariff): High-rate fraction: ', output_dict.get('value_241a'))
print('Space heating - main system 2 (electric off-peak tariff): Low-rate fraction: ', output_dict.get('value_241b'))
print('Space heating - main system 2 (electric off-peak tariff): High-rate cost: Fuel Price: ', output_dict.get('space_heating_main_system_2_high_rate_fuel_price'), '(£/kWh)')
print('Space heating - main system 2 (electric off-peak tariff): High-rate cost: Fuel Cost: ', output_dict.get('value_241c'), '(£/year)'),
print('Space heating - main system 2 (electric off-peak tariff): Low-rate cost: Fuel Price: ', output_dict.get('space_heating_main_system_2_low_rate_fuel_price'), '(£/kWh)')
print('Space heating - main system 2 (electric off-peak tariff): Low-rate cost: Fuel Cost: ', output_dict.get('value_241d'), '(£/year)')
print('Space heating - main system 2 (other fuel): Fuel Price: ', output_dict.get('space_heating_main_system_2_other_fuel_price'), '(£/kWh)')
print('Space heating - main system 2 (other fuel): Fuel Cost: ', output_dict.get('value_241e'), '(£/year)')

Space heating - main system 2 (electric off-peak tariff): High-rate fraction:  None
Space heating - main system 2 (electric off-peak tariff): Low-rate fraction:  None
Space heating - main system 2 (electric off-peak tariff): High-rate cost: Fuel Price:  None (£/kWh)
Space heating - main system 2 (electric off-peak tariff): High-rate cost: Fuel Cost:  None (£/year)
Space heating - main system 2 (electric off-peak tariff): Low-rate cost: Fuel Price:  None (£/kWh)
Space heating - main system 2 (electric off-peak tariff): Low-rate cost: Fuel Cost:  None (£/year)
Space heating - main system 2 (other fuel): Fuel Price:  None (£/kWh)
Space heating - main system 2 (other fuel): Fuel Cost:  None (£/year)


In [45]:
print('Space heating - secondary (electric off-peak tariff): High-rate fraction: ', output_dict.get('value_242a'))
print('Space heating - secondary (electric off-peak tariff): Low-rate fraction: ', output_dict.get('value_242b'))
print('Space heating - secondary (electric off-peak tariff): High-rate cost: Fuel Price: ', output_dict.get('space_heating_secondary_high_rate_fuel_price'), '(£/kWh)')
print('Space heating - secondary (electric off-peak tariff): High-rate cost: Fuel Cost: ', output_dict.get('value_242c'), '(£/year)'),
print('Space heating - secondary (electric off-peak tariff): Low-rate cost: Fuel Price: ', output_dict.get('space_heating_secondary_low_rate_fuel_price'), '(£/kWh)')
print('Space heating - secondary (electric off-peak tariff): Low-rate cost: Fuel Cost: ', output_dict.get('value_242d'), '(£/year)')
print('Space heating - secondary (other fuel): Fuel Price: ', output_dict.get('space_heating_secondary_other_fuel_price'), '(£/kWh)')
print('Space heating - secondary (other fuel): Fuel Cost: ', output_dict.get('value_242e'), '(£/year)')

Space heating - secondary (electric off-peak tariff): High-rate fraction:  None
Space heating - secondary (electric off-peak tariff): Low-rate fraction:  None
Space heating - secondary (electric off-peak tariff): High-rate cost: Fuel Price:  None (£/kWh)
Space heating - secondary (electric off-peak tariff): High-rate cost: Fuel Cost:  None (£/year)
Space heating - secondary (electric off-peak tariff): Low-rate cost: Fuel Price:  None (£/kWh)
Space heating - secondary (electric off-peak tariff): Low-rate cost: Fuel Cost:  None (£/year)
Space heating - secondary (other fuel): Fuel Price:  None (£/kWh)
Space heating - secondary (other fuel): Fuel Cost:  None (£/year)


In [63]:
print('Water heating (electric off-peak tariff): High-rate fraction: ', output_dict.get('value_243'))
print('Water heating (electric off-peak tariff): Low-rate fraction: ', output_dict.get('value_244'))
print('Water heating (electric off-peak tariff): High-rate cost: Fuel Price: ', output_dict.get('water_heating_high_rate_fuel_price'), '(£/kWh)')
print('Water heating (electric off-peak tariff): High-rate cost: Fuel Cost: ', output_dict.get('value_245'), '(£/year)'),
print('Water heating (electric off-peak tariff): Low-rate cost: Fuel Price: ', output_dict.get('water_heating_low_rate_fuel_price'), '(£/kWh)')
print('Water heating (electric off-peak tariff): Low-rate cost: Fuel Cost: ', output_dict.get('value_246'), '(£/year)')
print('Water heating (other fuel): Fuel Price: ', output_dict.get('water_heating_other_fuel_price'), '(£/kWh)')
print('Water heating (other fuel): Fuel Cost: ', '{:.2f}'.format(output_dict.get('value_247')), '(£/year)')

Water heating (electric off-peak tariff): High-rate fraction:  None
Water heating (electric off-peak tariff): Low-rate fraction:  None
Water heating (electric off-peak tariff): High-rate cost: Fuel Price:  None (£/kWh)
Water heating (electric off-peak tariff): High-rate cost: Fuel Cost:  None (£/year)
Water heating (electric off-peak tariff): Low-rate cost: Fuel Price:  None (£/kWh)
Water heating (electric off-peak tariff): Low-rate cost: Fuel Cost:  None (£/year)
Water heating (other fuel): Fuel Price:  3.64 (£/kWh)
Water heating (other fuel): Fuel Cost:  112.22 (£/year)


In [65]:
print('Energy for instantaneous electric shower(s): Fuel Price: ', output_dict.get('electricity_unit_price'), '(£/kWh)')
print('Energy for instantaneous electric shower(s): Fuel Cost: ', output_dict.get('value_247a'), '(£/year)')
print('Space cooling: Fuel Price: ', output_dict.get('electricity_unit_price'), '(£/kWh)')
print('Space cooling: Fuel Cost: ', output_dict.get('value_248'), '(£/year)')
print('Pumps, fans and electric keep-hot: Fuel Price: ', output_dict.get('electricity_unit_price'), '(£/kWh)')
print('Pumps, fans and electric keep-hot: Fuel Cost: ', output_dict.get('value_249'), '(£/year)')
print('Energy for lighting: Fuel Price: ', output_dict.get('electricity_unit_price'), '(£/kWh)')
print('Energy for lighting: Fuel Cost: ', '{:.2f}'.format(output_dict.get('value_250')), '(£/year)')
print('Additional standing charges: ', output_dict.get('value_251'), '(£/year)')
print('Energy saving/generation technologies: ', output_dict.get('value_252'), '(£/year)')
print('Appendix Q items: ')
print('Total energy cost: ', '{:.2f}'.format(output_dict.get('value_255')), '(£/year)')

Energy for instantaneous electric shower(s): Fuel Price:  16.49 (£/kWh)
Energy for instantaneous electric shower(s): Fuel Cost:  0.0 (£/year)
Space cooling: Fuel Price:  16.49 (£/kWh)
Space cooling: Fuel Cost:  None (£/year)
Pumps, fans and electric keep-hot: Fuel Price:  16.49 (£/kWh)
Pumps, fans and electric keep-hot: Fuel Cost:  0.0 (£/year)
Energy for lighting: Fuel Price:  16.49 (£/kWh)
Energy for lighting: Fuel Cost:  87.04 (£/year)
Additional standing charges:  92 (£/year)
Energy saving/generation technologies:  0.0 (£/year)
Appendix Q items: 
Total energy cost:  648.00 (£/year)


### SAP Worksheet - 11a. SAP rating – Individual heating systems including micro-CHP

In [59]:
print('Energy cost deflator (value_256): ', output_dict.get('value_256'))
print('Energy cost factor (ECF) (value_257): ', output_dict.get('value_257'))
print('SAP rating (value_258): ', output_dict.get('value_258'))
print('SAP band: ', output_dict.get('sap_band'))

Energy cost deflator (value_256):  0.36
Energy cost factor (ECF) (value_257):  1.6088324891911796
SAP rating (value_258):  74
SAP band:  C
